In [1]:
from copy import deepcopy
from functools import cache
from os import listdir
import json
import pickle
from datetime import datetime

from scipy.special import softmax
import numpy as np
import math
from itertools import combinations

from typing import Any, List, Tuple
import time

import torch
import dataclasses


# |Set root for GPUDrive import
import os
import sys
from pathlib import Path

from traitlets import default

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))


# |GPUDrive imports
from gpudrive.utils.config import load_config
from examples.CoDec_Research.code.simulation.construal_main import generate_baseline_data, generate_selected_construal_traj, \
                                                                    get_constral_heurisrtic_values, generate_all_construal_trajnval
from examples.CoDec_Research.code.gpuDrive_utils import get_gpuDrive_vars
from examples.CoDec_Research.code.analysis.evaluate_construal_actions import evaluate_construals, get_best_construals_likelihood
from examples.CoDec_Research.code.simulation.simulation_functions import simulate_selected_construal_policies



RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

2025-05-05 13:32:41.449747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746466361.513885     921 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746466361.534569     921 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746466361.674137     921 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746466361.674226     921 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746466361.674228     921 computation_placer.cc:177] computation placer alr

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

In [2]:

### Parameters for Inference ###
construal_selection_logic = "heuristic_1" # Set Construal Selection Logic. 
                                          # Options: "log_likelihood", "heuristic_1", "heuristic_2", "heuristic_3"
heuristic_params = {"dist_ego": 0.5} # Parameters for weighing the heuristics

### Specify Environment Configuration ###

# |Location to store (and find pre-computed) simulation results
simulation_results_path = "examples/CoDec_Research/results/simulation_results/"

# |Location to store simulation results
out_dir = "examples/CoDec_Research/results/simulation_results/"

# |Model Config (on which model was trained)
training_config = load_config("examples/experimental/config/reliable_agents_params")

# |Set scenario path
dataset_path = 'data/processed/construal'

# |Set simulator config
max_agents = training_config.max_controlled_agents   # Get total vehicle count
num_parallel_envs = 2
total_envs = 4
device = "cpu" # cpu just because we're in a notebook
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# |Set construal config
construal_size = 1
observed_agents_count = max_agents - 1      # Agents observed except self (used for vector sizes)
sample_size = 1                             # Number of samples to calculate expected utility of a construal

# |Other changes to variables
training_config.max_controlled_agents = 1    # Control only the first vehicle in the environment
total_envs = min(total_envs, len(listdir(dataset_path)))


### Instantiate Variables ###

env_config, train_loader, env, env_multi_agent, sim_agent = get_gpuDrive_vars(
                                                                                training_config = training_config,
                                                                                device = device,
                                                                                num_parallel_envs = num_parallel_envs,
                                                                                dataset_path = dataset_path,
                                                                                max_agents = max_agents,
                                                                                total_envs = total_envs,
                                                                                sim_agent_path= "daphne-cornelisse/policy_S10_000_02_27",
                                                                            )


In [3]:

### Select Construals for Baseline Data ###
scene_constr_dict = None
if construal_selection_logic == "heuristic_1":
    # |Select construals based on heuristic 1 (distance from ego)

    #2# |Generate Construal Execution Values through simulator sampling
    default_values = None

    #2# |Check if saved data is available
    simulation_results_files = [simulation_results_path+fl_name for fl_name in listdir(simulation_results_path)]
    for srFile in simulation_results_files:
        if "construal_vals" in srFile:
            with open(srFile, 'rb') as opn_file:
                default_values = pickle.load(opn_file)
        else:
            continue

    if default_values is None:
        default_values, traj_obs, ground_truth = generate_all_construal_trajnval(out_dir=out_dir,
                                                                                    sim_agent=sim_agent,
                                                                                    observed_agents_count=observed_agents_count,
                                                                                    construal_size=construal_size,
                                                                                    num_parallel_envs=num_parallel_envs,
                                                                                    max_agents=max_agents,
                                                                                    sample_size=sample_size,
                                                                                    device=device,
                                                                                    train_loader=train_loader,
                                                                                    env=env,
                                                                                    env_multi_agent=env_multi_agent,
                                                                                    generate_animations=False)

    #2# |Generate Construal Heuristic Values (Heuristic 1: Distance from ego)
    heuristic_values = get_constral_heurisrtic_values(env, train_loader, default_values, heuristic=1, heuristic_params=heuristic_params)

    #2# |Sample from construal values
    def sample_construals(heuristic_values: dict, sample_count: int) -> dict:
        """
        Sample construals based on heuristic values.
        """
        sampled_construals = {}
        for scene_name, construal_info in heuristic_values.items():
            constr_indices, constr_values = zip(*construal_info.items())
            constr_values_softmax = torch.nn.functional.softmax(torch.tensor(constr_values), dim=0)
            sampled_indices = torch.multinomial(constr_values_softmax, num_samples=sample_count, \
                                                               replacement=False).tolist()
            sampled_construals[scene_name] = {constr_indices[i]: constr_values[i] for i in sampled_indices}

        return sampled_construals
    
    scene_constr_dict = sample_construals(heuristic_values, sample_count=2)




Processing Waymo batches:   0%|          | 0/2 [00:00<?, ?it/s]

Indices of all moving vehicles (by scene):  [(0, 1, 5, 9, 10, 42, 43), (0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17)]
	sample  1
		Step: 90
Processed masks:  ((0,), (0,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((1,), (1,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((5,), (2,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((9,), (3,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((10,), (4,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((42,), (5,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((43,), (7,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((), (8,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((), (9,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((), (10,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90
Processed masks:  ((), (11,)) , with values: [1.0, 1.0]
	s

Processing Waymo batches:  50%|█████     | 1/2 [01:23<01:23, 83.02s/it]


Processed masks:  ((), ()) , with values: [1.0, 1.0]
Indices of all moving vehicles (by scene):  [(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20), (0, 7, 59)]
	sample  1
		Step: 91
Processed masks:  ((0,), (0,)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((1,), (7,)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((2,), (59,)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((3,), ()) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((4,), ()) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((5,), ()) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((6,), ()) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((7,), ()) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((8,), ()) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((9,), ()) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed mask

Processing Waymo batches: 100%|██████████| 2/2 [02:55<00:00, 87.87s/it]


Processed masks:  ((), ()) , with values: [1.0, 1.0]
Baseline data saved to:  examples/CoDec_Research/results/simulation_results/construal_vals_2025-05-04 14:06:12.891818.pickle


Baseline data saved to:  examples/CoDec_Research/results/simulation_results/all_constr_obs_2025-05-04 14:06:12.892199.pickle
Baseline data saved to:  examples/CoDec_Research/results/simulation_results/ground_truth_2025-05-04 14:06:12.924887.pickle


Processing Waymo batches: 100%|██████████| 2/2 [00:00<00:00,  5.10it/s]


In [4]:
### Generate Trajectories For Selected Construals (Baseline Data on Which to Perform Inference) ###
state_action_pairs = None

# |Check if saved data is available
simulation_results_files = [simulation_results_path+fl_name for fl_name in listdir(simulation_results_path)]
for srFile in simulation_results_files:
    if "baseline_state_action_pairs" not in srFile:
        continue
    with open(srFile, 'rb') as opn_file:
        state_action_pairs = pickle.load(opn_file)

if state_action_pairs is None:
    state_action_pairs = generate_baseline_data(out_dir=out_dir,
                                                sim_agent=sim_agent,
                                                num_parallel_envs=num_parallel_envs,
                                                max_agents=max_agents,
                                                sample_size=1,
                                                device=device,
                                                train_loader=train_loader,
                                                env=env,
                                                env_multi_agent=env_multi_agent,
                                                observed_agents_count=observed_agents_count,
                                                construal_size=construal_size,
                                                selected_construals=scene_constr_dict,
                                                generate_animations=False)


Processing Waymo batches:   0%|          | 0/2 [00:00<?, ?it/s]

	sample  1
		Step: 90
Processed masks:  ((1,), (13,)) , with values: [1.0, 1.0]
	sample  1
		Step: 90

Processing Waymo batches:  50%|█████     | 1/2 [00:09<00:09,  9.34s/it]


Processed masks:  ((42,), (15,)) , with values: [1.0, 1.0]
	sample  1
		Step: 91
Processed masks:  ((0,), (7,)) , with values: [1.0, 1.0]
	sample  1
		Step: 91

Processing Waymo batches: 100%|██████████| 2/2 [00:18<00:00,  9.38s/it]


Processed masks:  ((14,), (59,)) , with values: [1.0, 1.0]


Baseline data saved to:  examples/CoDec_Research/results/simulation_results/baseline_state_action_pairs_2025-05-04 14:06:32.122365.pickle


In [5]:
### Compute Construal Log Likelihoods ###
construal_action_likelihoods = None

# |Check if saved data is available
simulation_results_files = [simulation_results_path+fl_name for fl_name in listdir(simulation_results_path)]
for srFile in simulation_results_files:
    if "log_likelihood_measures" not in srFile:
        continue
    with open(srFile, 'rb') as opn_file:
        construal_action_likelihoods = pickle.load(opn_file)

construal_action_likelihoods = evaluate_construals(state_action_pairs, construal_size, sim_agent, out_dir, device=device)

# |Clear memory for large variable, once it has served its purpose
del state_action_pairs




Processing Scene:  tfrecord-00101-of-01000_71
Processing baseline construal (1,) against construal (0,), sample 0
Processing baseline construal (1,) against construal (1,), sample 0
Processing baseline construal (1,) against construal (5,), sample 0
Processing baseline construal (1,) against construal (9,), sample 0
Processing baseline construal (1,) against construal (10,), sample 0
Processing baseline construal (1,) against construal (42,), sample 0
Processing baseline construal (1,) against construal (43,), sample 0
Processing baseline construal (1,) against construal (), sample 0
Processing baseline construal (42,) against construal (0,), sample 0
Processing baseline construal (42,) against construal (1,), sample 0
Processing baseline construal (42,) against construal (5,), sample 0
Processing baseline construal (42,) against construal (9,), sample 0
Processing baseline construal (42,) against construal (10,), sample 0
Processing baseline construal (42,) against construal (42,), sa

In [6]:

### Construal Inference ###
scene_constr_diff_dict = None

# |Check if saved data is available
simulation_results_files = [simulation_results_path+fl_name for fl_name in listdir(simulation_results_path)]
for srFile in simulation_results_files:
    if "highest_construal_dict_log_likelihood_diff" in srFile:
        with open(srFile, 'rb') as opn_file:
            scene_constr_diff_dict = pickle.load(opn_file)
    elif "highest_construal_dict_log_likelihood" in srFile:
        with open(srFile, 'rb') as opn_file:
            scene_constr_dict = pickle.load(opn_file)
    else:
        continue       

if scene_constr_dict is None:
    scene_constr_dict = get_best_construals_likelihood(construal_action_likelihoods, out_dir)
if scene_constr_diff_dict is None:
    scene_constr_diff_dict = get_best_construals_likelihood(construal_action_likelihoods, out_dir, likelihood_key="log_likelihood_diff")





{(0,): {0: {'true_likelihoods': tensor([[1.0181e-04, 6.0829e-08, 1.3449e-08,  ..., 5.1721e-08, 6.8517e-03,
         2.4162e-01],
        [1.0569e-04, 6.1241e-08, 1.3225e-08,  ..., 5.3299e-08, 6.7690e-03,
         2.4183e-01],
        [7.8587e-05, 4.5211e-08, 6.9063e-09,  ..., 3.1431e-08, 4.9939e-03,
         2.4101e-01],
        ...,
        [5.6168e-07, 1.0489e-05, 3.3896e-04,  ..., 2.4512e-05, 4.8141e-08,
         3.4347e-07],
        [5.6168e-07, 1.0489e-05, 3.3896e-04,  ..., 2.4512e-05, 4.8141e-08,
         3.4347e-07],
        [5.6168e-07, 1.0489e-05, 3.3896e-04,  ..., 2.4512e-05, 4.8141e-08,
         3.4347e-07]], grad_fn=<CatBackward0>), 'pred_likelihoods': tensor([[3.4489e-04, 1.3761e-07, 7.1528e-08,  ..., 1.2040e-07, 8.6843e-03,
         2.1321e-01],
        [3.5445e-04, 1.3758e-07, 6.9139e-08,  ..., 1.2252e-07, 8.5483e-03,
         2.1365e-01],
        [2.5858e-04, 1.0034e-07, 3.4917e-08,  ..., 7.0774e-08, 6.2946e-03,
         2.1411e-01],
        ...,
        [5.5600e-07, 1.

In [8]:
scene_constr_dict

{'tfrecord-00101-of-01000_71': {(1,): np.float64(0.9548024362736818),
  (42,): np.float64(0.613054930528357)},
 'tfrecord-00117-of-01000_240': {(13,): np.float64(0.7074301356948892),
  (15,): np.float64(0.8085967087013745)},
 'tfrecord-00143-of-01000_179': {(0,): np.float64(1.0),
  (14,): np.float64(0.8890687447519268)},
 'tfrecord-00144-of-01000_130': {(59,): np.float64(0.8713387289404813),
  (7,): np.float64(0.9128845467943143)}}

In [13]:
for scene_name_, scene_info_ in construal_action_likelihoods.items():
    print(f"Scene: {scene_name_}")
    for base_construal_name_, base_construal_info_ in scene_info_.items():
        for test_construal_name_, test_construal_info_ in base_construal_info_.items():
            for sample_num_, sample_info_ in test_construal_info_.items():
                print(f"Base Construal: {base_construal_name_}, Test Construal: {test_construal_name_}, Sample: {sample_num_}")
                print(f"Log Likelihood: {sample_info_['log_likelihood']}")
                print(f"Log Likelihood Diff: {sample_info_['log_likelihood_diff']}")

Scene: tfrecord-00101-of-01000_71
Base Construal: (1,), Test Construal: (0,), Sample: 0
Log Likelihood: 54.592159271240234
Log Likelihood Diff: -7.497371196746826
Base Construal: (1,), Test Construal: (1,), Sample: 0
Log Likelihood: 47.094825744628906
Log Likelihood Diff: -3.6954959341528593e-06
Base Construal: (1,), Test Construal: (5,), Sample: 0
Log Likelihood: 52.114173889160156
Log Likelihood Diff: -5.019373416900635
Base Construal: (1,), Test Construal: (9,), Sample: 0
Log Likelihood: 54.6717414855957
Log Likelihood Diff: -7.576954364776611
Base Construal: (1,), Test Construal: (10,), Sample: 0
Log Likelihood: 54.5254020690918
Log Likelihood Diff: -7.4306182861328125
Base Construal: (1,), Test Construal: (42,), Sample: 0
Log Likelihood: 54.592159271240234
Log Likelihood Diff: -7.497371196746826
Base Construal: (1,), Test Construal: (43,), Sample: 0
Log Likelihood: 54.592159271240234
Log Likelihood Diff: -7.497371196746826
Base Construal: (1,), Test Construal: (), Sample: 0
Log Li